In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

## HMM 

Given that there are three speakers in the room, we should set $H = 3$, making $A$ a $3 \times 3$ matrix, and given the same seven phonemes from the previous exercise, we should set $V = 7$, making $B$ a $3 \times 7$ matrix.

A reasonable assumption is to set $\vec{a} = [\frac{1}{3}, \frac{1}{3}, \frac{1}{3}]^T$, a uniform distribution over the initial probabilities of each speaker starting. 

Note: 

$A_{ij} = P(h_{t + 1} = j | h_t = i)$, the row-major matrix notation.

$B_{ij} = P(v_t = j | h_t = i)$, again, row-major, unlike Barber's notation

## TODO: Change the link to the data below, and understand what the mapping does

In [ ]:
ALPHABET = ['A', 'o', 'e', 't', 'p', 'g', 'k']

data = open('data/hmm/speaker').read() 
transitions = list(zip(data, data[1:]))
mapping = {letter: index for (index, letter) in enumerate(ALPHABET)}
mapped_data = [mapping[v] for v in data]
mapped_transitions = [(mapping[a], mapping[b]) for (a, b) in transitions]

### Filtering

To implement filtering, we compute the forwards pass of $\alpha$ messages, using the following recursion:

$$ \alpha(h_t) = p(h_t | v_{1:t}) = p(v_t | h_t) \sum_{h_{t-1}} p(h_t | h_{t-1}) \alpha(h_{t-1})  $$

Which if we consider the factor graph, is just the product of messages from $v_t$ and $h_{t-1}$. The base case includes the messages from $v_1$ and the prior over $a$: $\alpha(h_1) = p(h_1)p(v_1 | h_1)$

## TODO: implement the alpha message. 

`alpha(a, A, B, mapped_data)` receives four parameters - the a vector, A and B matrices, and data mapped as indices. `alpha` should return the $\alpha(h_t)$ messages over the entire data - for every time step, the message (probability) for each possible value of $h_t$

In [ ]:
def alpha(a, A, B, mapped_data):
    # TODO: implement
    pass

### Parallel Smoothing
To implement parallel smoothing, we must also compute the backwards pass: 

$$ P(h_t, v_{1:T}) = P(h_t, v_{1:t}, v_{t+1:T}) = P(h_t | v_{1:t}) P(v_{t+1:T} | h_t) = \alpha(h_t) \beta(h_t) $$

Thus, defining $\beta(h_t) \equiv p(v_{t+1:T} | h_t)$, we compute it recursively as:

$$ \beta(h_{t-1}) = \sum_{h_t} p(v_t | h_t) P(h_t | h_{t-1}) \beta(h_t) $$

With the base case of $\beta(h_T) = 1$. Again, if we think of the factor graph, the message $\beta(h_t)$ is simply the mesage from $h_{t+1}$ to $h_t$

## TODO: implement the beta message.
`beta(a, A, B, mapped_data)` receives the same parameters as `alpha`, and should return the $\beta(h_t)$ messages over the entire data - for every time step, the message (probability) for each possible value of $h_t$

In [ ]:
def beta(a, A, B, mapped_data):
    # TODO: implement me
    pass

### Smoothed Posterior
The smoothed posterior is hence given by:

$$ P(h_t | v_{1:T}) \equiv \gamma(h_t) = \frac{\alpha(h_t) \beta(h_t)}{\sum_{h_t} \alpha(h_t) \beta(h_t)} $$

Which if we think in terms of the factor graph represents the marginal over $h_t$, the locally normalized product of all incoming messages.

## TODO: implement the gamma message.
`gamma(alpha_messages, beta_messages)` receives two arguments - the list of $\alpha(h_t)$ messages, and the list of $\beta(h_t)$ messages - and returns a the list of normalized, smoothed posteriors over the entire data set - for each data point, the posterior over all values of $h_t$ at that point

In [ ]:
def gamma(alpha_messages, beta_messages):
    # TODO: implement me
    pass

### Pairwise Marginal

Following the explicit definition, we use the messages to compute the pairwise marginal:

$$ P(h_t, h_{t+1} | v_{1:T}) \propto \alpha(h_t) P(h_{t+1} | h_t) p(v_{t+1} | h_{t+1}) \beta(h_{t+1}) $$

The result should be an $H \times H$ matrix, in which $M_{ij} = P(h_t = i, h_{t+1} = j | v_{1:T})$

This is easier to think of in terms of a junction tree - it represents the posterior over the clique $\{h_t, h_{t+1}\}$

## TODO: implement the pairwise marginals
`pairwise_marginals(alpha_messages, beta_messages, mapped_data)` receives three arguments, which should be fairly obvious by now, and returns a list of the pairwise marginls. Note that since this list of of marginals over pairs, it should have one fewer element than the length of the $\alpha(h_t)$ mesages. 

It is not obvious how the output of this function should be structured - I chose the output format decribed above, but you could make other choices that would be reasonable

In [ ]:
def pairwise_marginals(alpha_messages, beta_messages, mapped_data):
    # TODO: implement me

### EM Implementation
Using the functions defined above, we can now go into doing what we actually came here to do - implement the EM algorithm. The E-step is trivial:

Since this is simply the smoothed inference, I'm leaving the implementation here:

In [ ]:
def E_step(a, A, B, mapped_data):
    alpha_messages = alpha(a, A, B, mapped_data)
    beta_messages = beta(a, A, B, mapped_data)
    
    return gamma(alpha_messages, beta_messages), \
            pairwise_marginals(alpha_messages, beta_messages, mapped_data)

The M-step requires implementing the formulae provided by Barber:

$$ a_i^{new} \equiv P^{new}(h_1 = i) = \frac{1}{N} \sum_{n=1}^{N} p^{old}(h_1 = i | v^n)  $$

$$ A_{ij}^{new} \equiv P^{new}(h_{t+1} = j | h_t = i) \propto \sum_{n=1}^{N} \sum_{t=1}^{T_n -1} p^{old}(h_t = i, h_{t+1} = j | v^n) = \frac{\sum_{n=1}^{N} \sum_{t=1}^{T_n -1} p^{old}(h_t = i, h_{t+1} = j | v^n)}{\sum_j \sum_{n=1}^{N} \sum_{t=1}^{T_n -1} p^{old}(h_t = i, h_{t+1} = j | v^n)} $$


$$ B_{ij}^{new} \equiv P^{new}(v_t = j | h_t = i) \propto \sum_{n=1}^{N} \sum_{t=1}^{T_n -1} \mathbb{I}[v_t^n = j] p^{old}(h_t = i | v^n) $$

Note that in this case, we currently only have a single data point, so $n = 1$, and the summation over $n$ doesn't exist. Also note that all the $P^{old}$ are, infact, the quantities we computed in the E-step:

## TODO: implement the M_step

The three equations above look a little ugly, but given the node posteriors and pair marginals, as well as the mapped data matrix provided below, it is less scary than it looks like. `M_step` should return three results - the new values for $a$, $A$, and $B$.

In [ ]:
def M_step(node_posteriors, pair_marginals, mapped_data_matrix):
    # TODO: implement me
    a = None
    A = None
    B = None
    return a, A, B

## TODO: pick good initialization values for a, A, and B

In [ ]:
H = 3
V = 7

a = np.zeros((H,)) # this is probably not correct
A = np.zeros((H, H)) # this is probably not correct
B = np.zeros((H, V)) # this is probably not correct


def EM(a, A, B, mapped_data):
    mapped_data_matrix = np.zeros((len(mapped_data), V))
    for t, v_t in enumerate(mapped_data):
        mapped_data_matrix[t, v_t] = 1
        
    for i in range(3):
        node_posteriors, pair_marginals = E_step(a, A, B, mapped_data)
        a, A, B = M_step(node_posteriors, pair_marginals, mapped_data_matrix)
    
    return a, A, B
   
new_a, new_A, new_B = EM(a, A, B, mapped_data)
new_a, new_A, new_B

## TODO: implement the stackplot 
Displaying the probability of each speaker at each point in time